In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.fftpack as fft
from astropy.io import fits
from scipy import optimize
import poppy
import gpipsfs
from scipy import signal
import os
import pytz
import time
import re

In [ ]:
file1 =  '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160229/aored_When_2016.2.29_5.33.19_poldm_phase.fits'
file2 =  '/Users/MelisaT/Documents/Research/GPIDomeSeeing/data/Reduced/20160227/aored_When_2016.2.27_0.2.8_poldm_phase.fits'

# Analysis Functions

In [ ]:
def process_phase(filename):

    # Import fits
    hdulist = fits.open(filename,memmap=True)
    phase = hdulist[0].data.astype('float')
    avg_phase = np.mean(phase,axis=0) 

    # Estimate static aberrations
    z_basis = poppy.zernike.zernike_basis_faster(nterms= 6, npix = 48)
    z_coeff = poppy.zernike.opd_expand_nonorthonormal(avg_phase,aperture=M1,nterms=6)
    thin_lens = np.sum(z_coeff[:,None,None]*z_basis[:,:,:],axis=0)

    c_phase = (phase - thin_lens[None,:,:])*ap
    c_phase[np.isnan(c_phase)]=0.
    
    return c_phase

In [ ]:
def radialProfile(image, center=None):
    """
    Calculate the avearge radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    
    """
    ## Calculate the indices from the image
    y,x = np.indices((image.shape)) # first determine radii of all pixels
    
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])
     
    r = np.hypot(x - center[0], y - center[1]).astype(np.int) 
    
    n = np.bincount(r.ravel())
    sy = np.bincount(r.ravel(), image.ravel())
    mean = sy/n
    
    return mean

In [ ]:
def sp_power_spec(phase):    
    
    timesteps, phx, phy = phase.shape 
    phFT = np.zeros((timesteps,phx,phy), dtype=complex)
    for t in np.arange(timesteps):
        phFT[t,:,:] = fft.fftshift(fft.fft2(phase[t,:,:]))/ap.sum() # what happens if you remove the shift??????
    print('Done with FT')
    
    # compute 2d psd cube
    psd2D = np.zeros((timesteps, phx, phy),dtype=float)
    for k in np.arange(phx):
        for l in np.arange(phy):
            psd2D[:,k,l] = np.square(np.abs(phFT[:,k,l]))
    
    avg_psd2D = np.sum(psd2D, axis=0)
    print('Done with PSD')    
    
    # compute radial average of 2d psd cube and frequency
    avg_psd1D =  radialProfile(avg_psd2D)
    
    return avg_psd1D

In [ ]:
def sri_power_spec(mode, interval_length = 1024):
    
    total_len = len(mode)
    per_len = interval_length
    
    # Define indices
    num_intervals = np.floor(total_len/per_len).astype(int)
    start_indices = np.arange(num_intervals,dtype=int)
    ind = np.arange(per_len, dtype=float)
    
    # blackman window
    window = 0.42 - 0.50 * np.cos(2 * np.pi * ind/(per_len-1)) + 0.08 * np.cos(4 * np.pi * ind/(per_len-1)) 
    
    # psd calculation
    psd = np.zeros(per_len)
    for a in np.arange(num_intervals):
        this_start = start_indices[a]
        psd = psd + np.abs(fft.fftshift(fft.fft(mode[this_start:(this_start+per_len)]*window))/per_len)**2
        
    psd = psd/num_intervals
    win2 = window**2
    psd = psd * per_len/win2.sum()
    
    return psd

In [ ]:
def temp_power_spec(Y):
    
    n = len(Y)
    w = signal.blackman(n)
    P = np.fft.rfft(Y*w)
    norm = 2.0/w.sum()
    P = P * norm

    P2 = np.square(np.abs(P))
    #k = np.fft.rfftfreq(n,dT)
    
    return P2

In [ ]:
def linear_fit(k,Y,low_b,up_b):

    par = np.polyfit(np.log10(k[(k>low_b) & (k<up_b)]), np.log10(Y[(k>low_b) & (k<up_b)]), 1)
    slope = par[0]
    intercept = par[1]
    print(slope,intercept)
    return slope, intercept   

# Finding the PSD of modes

In [ ]:
n = 48
pscale = 0.18
phase = process_phase(file2)
timesteps, phx, phy = phase.shape 

# spatial frequncy grid
kx = fft.fftshift(fft.fftfreq(n,pscale))
ky = fft.fftshift(fft.fftfreq(n,pscale))
mg = np.meshgrid(kx,ky)
kr = np.sqrt(np.sum((m**2 for m in mg))) 

# obtain fourier modes
phFT = np.zeros((timesteps,phx,phy), dtype=complex)
for t in np.arange(timesteps):
    phFT[t,:,:] = fft.fftshift(fft.fft2(phase[t,:,:]))/(phx*phy)
print('Done with FT')
    
#compute 2d psd cube
psd2D = np.zeros((1024, phx, phy),dtype=float)
for k in np.arange(phx):
    for l in np.arange(phy):
        psd2D[:,k,l] = sri_power_spec(phFT[:,k,l])
        
varpsd = np.sum(psd2D, axis = 0)

In [ ]:
%matplotlib notebook

k = radialProfile(kr)
psd1D = radialProfile(varpsd)
slope, yint = linear_fit(k,psd1D,.33,1)

plt.loglog(k,psd1D)
plt.loglog(k,10**yint*k**slope,'rx')


In [ ]:
%matplotlib notebook

k = fft.fftshift(fft.fftfreq(1024,.001))
psd1D = psd2D[:,4,4]
smooth = 10**signal.savgol_filter(np.log10(psd1D), 101, 5)
slope, yint = linear_fit(k,smooth,2,30)

plt.loglog(smooth)
#plt.loglog(k,10**yint*k**slope,'rx')


# Measure PSD

In [ ]:
# GPI optical elements
ap = gpipsfs.GPI_Apodizer().sample(npix=48)
M1 = gpipsfs.GeminiPrimary().sample(npix=48)
n=48
pscale = .18

# frequency grid
kx = fft.fftshift(fft.fftfreq(n,2*np.pi/(pscale)))
ky = fft.fftshift(fft.fftfreq(n,2*np.pi/(pscale)))
mg = np.meshgrid(kx,ky)
kr = np.sqrt(np.sum((m**2 for m in mg))) 

k = radialProfile(kr)

In [ ]:
y_2D = process_phase(file1)
y_psd = sp_power_spec(y_2D)

In [ ]:
m, yint = linear_fit(k,y_psd,.33,2.2)

In [ ]:
y_1D = np.mean(y_2D,axis=(1,2))
y_psd = temp_power_spec(y_1D)
y_smoothed = 10**signal.savgol_filter(np.log10(y_psd), 101, 5)
kt = fft.rfftfreq(len(y_smoothed),.001)

In [ ]:
m, yint = linear_fit(kt,y_smoothed,5,30)

In [ ]:
bounds = (kt>5) & (kt<30)

In [ ]:
%matplotlib notebook
plt.loglog(kt,y_smoothed)
plt.loglog(kt[bounds],10**yint*kt[bounds]**m,'x')